In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

# 5 Series Temporales

Cualquier dato que se registre repetidamente en muchos puntos en el tiempo forma una serie temporal. Muchas series temporales tienen una frecuencia fija, es decir, los puntos de datos ocurren a intervalos regulares según alguna regla, como, por ejemplo, cada 15 segundos, cada 5 minutos o una vez al mes. Las series temporales también pueden ser irregulares, sin unidad de tiempo fija ni desfase entre unidades. El modo en que se marcan los datos de series temporales y se hace referencia a ellos depende de la aplicación, así que podríamos tener cualesquiera de los siguientes:

- Marcas temporales: Instantes específicos en el tiempo.
- Periodos fijos: Como, por ejemplo, el mes entero de enero de 2017, o todo
el año 2020.
- Intervalos de tiempo: Indicados por una marca temporal inicial y final. Los
periodos pueden considerarse como casos especiales de intervalos.
- Tiempo transcurrido o tiempo del experimento: Cada marca temporal es
una medida de tiempo relativa a un determinado momento inicial (por
ejemplo, el diámetro de una galleta que se hornea cada segundo desde que
es introducida en el horno), empezando por 0.

En pandas encontramos muchas herramientas y algoritmos de series
temporales. Se puede trabajar de una manera eficaz con grandes series
temporales, y segmentar, agregar y remuestrear series temporales irregulares y
de frecuencia fija. Algunas de estas herramientas son útiles para aplicaciones
financieras y económicas, pero, sin duda, se pueden utilizar perfectamente para
analizar datos de registros de servidor.

In [2]:
import numpy as np
import pandas as pd

## 5.1 Tipos de datos de fecha y hora y herramientas asociadas

La librería estándar de Python incluye tipos de datos para fechas y horas,
además de funcionalidad relacionada con el calendario. Los módulos `datetime`,
`time` y `calendar` son los lugares principales para empezar. El tipo
`datetime.datetime`, o simplemente `datetime`, se utiliza mucho:

In [3]:
from datetime import datetime
now = datetime.now()
now

datetime.datetime(2024, 7, 22, 16, 10, 51, 269902)

In [4]:
now.year, now.month, now.day

(2024, 7, 22)

El tipo `datetime` almacena la fecha y hora hasta el microsegundo, mientras
que datetime.timedelta, o simplemente timedelta, representa la diferencia de
tiempo entre dos objetos datetime:

In [5]:
delta = datetime(2011, 1, 7) - datetime(2008, 6, 24, 8, 15)
delta

datetime.timedelta(days=926, seconds=56700)

In [6]:
delta.days

926

In [7]:
delta.seconds

56700

Se puede sumar (o restar) un timedelta o varios a un objeto datetime para
obtener un nuevo objeto desplazado en el tiempo:

In [8]:
from datetime import timedelta
start = datetime(2011, 1, 7)
start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

In [9]:
start - 2 * timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

La siguiente tabla resume los tipos de datos del módulo `datetime`

**Tipo**: **Descripción**

`date`: Almacena la fecha del calendario (año, mes, día) utilizando el calendario gregoriano.

`time`: Almacena la hora del día como horas, minutos, segundos y microsegundos.

`Datetime`: Almacena la fecha y la hora.

`timedelta`: La diferencia entre dos valores datetime (como días, segundos y microsegundos).

`tzinfo`: Tipo básico para almacenar información de zona horaria.

**Conversión entre cadena de texto y datetime**

Se pueden formatear objetos `datetime` y objetos `Timestamp` de pandas (que
se explicaraá más adelante) como cadenas de texto utilizando str o el método `strftime`, pasando una especificación de formato:

In [10]:
stamp = datetime(2011, 1, 3)
str(stamp)

'2011-01-03 00:00:00'

In [11]:
stamp.strftime("%Y-%m-%d")

'2011-01-03'

La siguiente tabla ofrece una lista completa de códigos de formato:

**Tipo** : **Descripción**
    
`%Y`: año de cuatro dígitos.
    
`%y`: Año de dos dígitos.
    
`%m`: Mes de dos dígitos [01, 12].

`%d`: Día de los dígitos [01, 31]

`%H`: Hora (formato de 24 horas) [00, 23].

`%I`: Hora (formato de 12 horas) [01, 12].

`%M`: Minuto de dos dígitos [00, 59].

`%s`: Segundo [00, 61] (60 segundos, 61 para segundos bisiestos).

`%f`: Microsegundo como un entero, rellenado con ceros (de 000000 a 999999).

`%j`: Día del año como entero rellenado con ceros (de 001 a 336).

`%w`: Día de la semana como entero [0 (domingo), 6].

`%u`: Día de la semana como entero empezando por 1, donde 1 es lunes.

`%U`: Número de semana del año [00, 53]; el domingo se considera el primer día de la semana, y los días que preceden al primer domingo del año son la «semana 0».

`%W`: Número de semana del año [00, 53]; el lunes se considera el primer día de la semana, y los días que preceden al primer lunes del año son la «semana 0».

`%z`: Desfase de zona horaria UTC como +HHMM o –HHMM; queda vacío si no se es consciente de la zona horaria.

`%Z`: Nombre de la zona horaria como cadena de texto, o cadena de texto vacío si no hay zona horaria.

`%F`: Abreviatura de %Y-%m-%d (por ejemplo, 2012-04-18).

`%D`: Abreviatura de %m/%d/%Y (por ejemplo, 04/18/2012).

Se pueden utilizar muchos de los mismos códigos de formato para convertir
cadenas de texto en fechas utilizando `datetime.strptime` (pero algunos
códigos, como `%F`, no se pueden usar):

In [12]:
value = "2011-01-03"
datetime.strptime(value, "%Y-%m-%d")

datetime.datetime(2011, 1, 3, 0, 0)

In [13]:
datestrs = ["7/6/2011", "8/6/2011"]
time1= [datetime.strptime(x, "%m/%d/%Y") for x in datestrs]
time1

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

El método `datetime.strptime` es una forma de analizar una fecha con un
formato conocido. Pandas está normalmente orientado al trabajo con arrays de fechas, ya se utilicen como el índice de un eje o como una columna de un dataframe. El método `pandas.to_datetime` analiza muchos tipos distintos de representaciones de fecha. Los formatos de fecha estándares, como ISO 8601, se pueden analizar rápidamente:

In [14]:
datestrs = ["2011-07-06 12:00:00", "2011-08-06 00:00:00"]
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00'], dtype='datetime64[ns]', freq=None)

Tambien gestiona valores que se deberían considerar ausentes (None, cadena
de texto vacía, etc.):

In [15]:
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)

In [16]:
idx[2]

NaT

In [17]:
pd.isna(idx)

array([False, False,  True])

NaT (Not a Time) es el valor nulo de pandas para datos de marca temporal.

Los objetos datetime tienen también una serie de opciones de formato de
configuración regional para sistemas de otros países o idiomas. Por ejemplo, los nombres de meses abreviados serán diferentes en los sistemas alemán o francés comparados con los sistemas anglosajones:

**Tipo**: **Descripción**

`%a`: Nombre del día de la semana abreviado.

`%A`: Nombre del día de la semana completo.

`%b` : Nombre del mes abreviado.

`%B`: Nombre del mes completo.

`%c`: Fecha y hora completas (por ejemplo, en inglés 'Tue 01 May 2012 04:20:57 PM').

`%p`: Equivalente local de AM o PM.

`%x`: Fecha con formato adecuado a la localidad (por ejemplo, en los Estados Unidos, May 1, 2012 produce '05/01/2012').

`%X`: Hora con formato adecuado a la localidad (por ejemplo, '04:24:12 PM').

## 5.2 Fundamentos de las series temporales

Un tipo básico de objeto serie temporal en pandas es una serie indexada por
marcas temporales, que a menudo se representa fuera de pandas como una
cadena de texto Python o como un objeto `datetime`:

In [18]:
dates = [datetime(2011, 1, 2), datetime(2011, 1, 5),
         datetime(2011, 1, 7), datetime(2011, 1, 8),
         datetime(2011, 1, 10), datetime(2011, 1, 12)]
ts = pd.Series(np.random.standard_normal(6), index=dates)
ts

2011-01-02   -0.204708
2011-01-05    0.478943
2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
2011-01-12    1.393406
dtype: float64

Técnicamente, estos objetos `datetime` se han colocado en un
`DatetimeIndex`:

In [19]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

Igual que cualquier otro objeto Series, las operaciones aritméticas entre
distintas series temporales indexadas de distinta manera se alinean según las fechas:

In [20]:
ts[::2]

2011-01-02   -0.204708
2011-01-07   -0.519439
2011-01-10    1.965781
dtype: float64

In [21]:
ts + ts[::2]

2011-01-02   -0.409415
2011-01-05         NaN
2011-01-07   -1.038877
2011-01-08         NaN
2011-01-10    3.931561
2011-01-12         NaN
dtype: float64

Recordemos que ts[::2] selecciona cada segundo elemento de ts.
En pandas las marcas temporales se almacenan utilizando el tipo de datos
datetime64 de NumPy a la resolución de nanosegundo:

In [22]:
ts.index.dtype

dtype('<M8[ns]')

Los valores escalares de un `DatetimeIndex` son objetos `Timestamp` de
pandas:

In [23]:
ts

2011-01-02   -0.204708
2011-01-05    0.478943
2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
2011-01-12    1.393406
dtype: float64

In [24]:
stamp = ts.index[0]
stamp

Timestamp('2011-01-02 00:00:00')

Un objeto `pandas.Timestamp` se puede sustituir en la mayoría de los sitios en
los que se utilizaría un objeto `datetime`. Pero a la inversa es imposible, porque
`pandas.Timestamp` puede almacenar datos con precisión de nanosegundo,
mientras que `datetime` solo almacena hasta microsegundos. Además,
`pandas.Timestamp` puede almacenar informacion de frecuencias (si es que la
hay) y comprende cómo realizar conversiones de zona horaria y otros tipos de
manejos de tiempo.

**Indexación, selección y creación de subconjuntos**

Las series temporales se comportan como cualquier otro objeto Series cuando
se están indexando y seleccionando datos basados en la etiqueta:

In [25]:
ts

2011-01-02   -0.204708
2011-01-05    0.478943
2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
2011-01-12    1.393406
dtype: float64

In [26]:
stamp = ts.index[2]
stamp

Timestamp('2011-01-07 00:00:00')

In [27]:
ts[stamp]

np.float64(-0.5194387150567381)

Por comodidad se puede pasar también una cadena que puede interpretarse
como una fecha:

In [28]:
ts["2011-01-10"]

np.float64(1.9657805725027142)

Para series temporales más largas, se puede pasar un año o solo un año y un
mes para seleccionar fácilmente segmentos de datos

In [29]:
longer_ts = pd.Series(np.random.standard_normal(1000),
                      index=pd.date_range("2000-01-01", periods=1000))
longer_ts

2000-01-01    0.092908
2000-01-02    0.281746
2000-01-03    0.769023
2000-01-04    1.246435
2000-01-05    1.007189
                ...   
2002-09-22    0.930944
2002-09-23   -0.811676
2002-09-24   -1.830156
2002-09-25   -0.138730
2002-09-26    0.334088
Freq: D, Length: 1000, dtype: float64

In [30]:
longer_ts["2001"]

2001-01-01    1.599534
2001-01-02    0.474071
2001-01-03    0.151326
2001-01-04   -0.542173
2001-01-05   -0.475496
                ...   
2001-12-27    0.057874
2001-12-28   -0.433739
2001-12-29    0.092698
2001-12-30   -1.397820
2001-12-31    1.457823
Freq: D, Length: 365, dtype: float64

Aquí, la cadena de texto “2001” se interpreta como un año, así que se
selecciona ese periodo de tiempo. Esto también funciona si se especifica el mes:

In [31]:
longer_ts["2001-05"]

2001-05-01   -0.622547
2001-05-02    0.936289
2001-05-03    0.750018
2001-05-04   -0.056715
2001-05-05    2.300675
                ...   
2001-05-27    0.235477
2001-05-28    0.111835
2001-05-29   -1.251504
2001-05-30   -2.949343
2001-05-31    0.634634
Freq: D, Length: 31, dtype: float64

Segmentar con objetos `datetime` también funciona:

In [32]:
ts

2011-01-02   -0.204708
2011-01-05    0.478943
2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
2011-01-12    1.393406
dtype: float64

In [33]:
ts[datetime(2011, 1, 7):]

2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
2011-01-12    1.393406
dtype: float64

In [34]:
ts[datetime(2011, 1, 7):datetime(2011, 1, 10)]

2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
dtype: float64

Como la mayoría de los datos de series temporales está ordenada
cronológicamente, se puede segmentar con marcas temporales no contenidas en
una serie temporal para realizar la consulta de un rango:

In [35]:
ts

2011-01-02   -0.204708
2011-01-05    0.478943
2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
2011-01-12    1.393406
dtype: float64

In [36]:
ts["2011-01-06":"2011-01-11"]

2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
dtype: float64

In [37]:
ts

2011-01-02   -0.204708
2011-01-05    0.478943
2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
2011-01-12    1.393406
dtype: float64

Como antes, se puede pasar una fecha de cadena de texto, un datetime o una
marca temporal. Recordemos que segmentar de este modo produce vistas según
la serie temporal de origen, igual que cuando se fragmentan arrays NumPy. Esto
significa que no se copian datos, y que las modificaciones de la segmentación se
verán reflejadas en los datos originales. 

Hay un método de instancia equivalente, truncate, que divide un objeto
Series entre dos fechas:

In [38]:
ts.truncate(after="2011-01-09")

2011-01-02   -0.204708
2011-01-05    0.478943
2011-01-07   -0.519439
2011-01-08   -0.555730
dtype: float64

Cuando se utiliza el argumento `after`, pandas elimina todos los datos después de la fecha especificada.

Todo esto se aplica igualmente a objetos DataFrame, indexando según sus
filas:

In [39]:
dates = pd.date_range("2000-01-01", periods=100, freq="W-WED")
dates

DatetimeIndex(['2000-01-05', '2000-01-12', '2000-01-19', '2000-01-26',
               '2000-02-02', '2000-02-09', '2000-02-16', '2000-02-23',
               '2000-03-01', '2000-03-08', '2000-03-15', '2000-03-22',
               '2000-03-29', '2000-04-05', '2000-04-12', '2000-04-19',
               '2000-04-26', '2000-05-03', '2000-05-10', '2000-05-17',
               '2000-05-24', '2000-05-31', '2000-06-07', '2000-06-14',
               '2000-06-21', '2000-06-28', '2000-07-05', '2000-07-12',
               '2000-07-19', '2000-07-26', '2000-08-02', '2000-08-09',
               '2000-08-16', '2000-08-23', '2000-08-30', '2000-09-06',
               '2000-09-13', '2000-09-20', '2000-09-27', '2000-10-04',
               '2000-10-11', '2000-10-18', '2000-10-25', '2000-11-01',
               '2000-11-08', '2000-11-15', '2000-11-22', '2000-11-29',
               '2000-12-06', '2000-12-13', '2000-12-20', '2000-12-27',
               '2001-01-03', '2001-01-10', '2001-01-17', '2001-01-24',
      

Vamos a crear el DataFrame:

In [40]:
long_df = pd.DataFrame(np.random.standard_normal((100, 4)),
                       index=dates,
                       columns=["Colorado", "Texas",
                                "New York", "Ohio"])
long_df.loc["2001-05"]

,Colorado,Texas,New York,Ohio
2001-05-02,-0.006045,0.490094,-0.277186,-0.707213
2001-05-09,-0.560107,2.735527,0.927335,1.513906
2001-05-16,0.538600,1.273768,0.667876,-0.969206
2001-05-23,1.676091,-0.817649,0.050188,1.951312
2001-05-30,3.260383,0.963301,1.201206,-1.852001


**Series temporales con índices duplicados**

En algunas aplicaciones se pueden producir varias observaciones de datos
que entren dentro de una determinada marca temporal. Aquí tenemos un ejemplo:

In [41]:
dates = pd.DatetimeIndex(["2000-01-01", "2000-01-02", "2000-01-02",
                          "2000-01-02", "2000-01-03"])
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-02', '2000-01-02',
               '2000-01-03'],
              dtype='datetime64[ns]', freq=None)

In [42]:
dup_ts = pd.Series(np.arange(5), index=dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

Podemos decir que el índice no es único comprobando su propiedad
is_unique:

In [43]:
dup_ts.index.is_unique

False

Indexar dentro de esta serie temporal producirá ahora valores escalares o bien
segmentos, dependiendo de si una marca temporal está duplicada:

In [44]:
dup_ts["2000-01-03"]  # No duplicado

np.int64(4)

In [45]:
dup_ts["2000-01-02"]  # duplicado

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64

Supongamos que queremos agregar los datos que tienen marcas temporales
no únicas. Una forma de hacerlo es utilizando groupby y pasando level=0 (el
único nivel existente):

In [46]:
grouped = dup_ts.groupby(level=0)

Se formaran tres grupos diferentes que pandas guarda como `chunks`. Imprima grouped sin usar agregaciones:

In [47]:
grouped.mean()

2000-01-01    0.0
2000-01-02    2.0
2000-01-03    4.0
dtype: float64

In [48]:
grouped.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

## 5.3 Rangos de fechas, frecuencias y desplazamiento

Las series temporales genéricas en pandas se suponen irregulares, es decir, no
tienen una frecuencia fija. Para muchas aplicaciones esto es suficiente, pero a
veces es deseable trabajar en relación con una frecuencia fija, como diaria,
mensual o cada 15 minutos, incluso si ello significa introducir valores ausentes
en una serie temporal. Afortunadamente, pandas incluye un juego completo de
frecuencias y herramientas estándares de series temporales para remuestrear
, inferir frecuencias y generar rangos de fechas de frecuencia fija.
Por ejemplo, es posible convertir la serie temporal de muestra en una frecuencia
diaria fija llamando a `resample`:

In [49]:
ts

2011-01-02   -0.204708
2011-01-05    0.478943
2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-10    1.965781
2011-01-12    1.393406
dtype: float64

In [50]:
resampler = ts.resample("D")
# La cadena de texto “D” se interpreta como frecuencia diaria.
resampler

**Imprimir sin agregaciones**

In [51]:
resampled_ts = resampler.asfreq()
resampled_ts

2011-01-02   -0.204708
2011-01-03         NaN
2011-01-04         NaN
2011-01-05    0.478943
2011-01-06         NaN
2011-01-07   -0.519439
2011-01-08   -0.555730
2011-01-09         NaN
2011-01-10    1.965781
2011-01-11         NaN
2011-01-12    1.393406
Freq: D, dtype: float64

La cadena de texto “D” se interpreta como frecuencia diaria.

**Generación de rangos de fechas**

Aunque ya se ha utilizado anteriormente sin dar una explicación,
`pandas.date_range` es responsable de generar un DatetimeIndex con una
longitud indicada según una determinada frecuencia:

In [52]:
index = pd.date_range("2012-04-01", "2012-06-01")
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

De forma predeterminada, `pandas.date_range` genera marcas temporales
diarias. Si se pasa solamente una fecha de inicio o de fin, se deben pasar el número de periodos que se deseen generar:

In [53]:
pd.date_range(start="2012-04-01", periods=20)


DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

In [54]:
pd.date_range(end="2012-06-01", periods=20)

DatetimeIndex(['2012-05-13', '2012-05-14', '2012-05-15', '2012-05-16',
               '2012-05-17', '2012-05-18', '2012-05-19', '2012-05-20',
               '2012-05-21', '2012-05-22', '2012-05-23', '2012-05-24',
               '2012-05-25', '2012-05-26', '2012-05-27', '2012-05-28',
               '2012-05-29', '2012-05-30', '2012-05-31', '2012-06-01'],
              dtype='datetime64[ns]', freq='D')

Las fechas de inicio y fin definen estrictos límites para el índice de fecha
generado. Por ejemplo, si quisiéramos un índice de fecha que contuviera el
último día laborable de cada mes, pasaríamos la frecuencia `“BM”` (fin de mes
laborable; la tabla a continuación ofrece un listado completo de frecuencias), y solo se
incluirían las fechas que entraran dentro de ese intervalo de fechas:

In [55]:
pd.date_range("2000-01-01", "2000-12-01", freq="BM")

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\3742551278.py:1: FutureWarning: 'BM' is deprecated and will be removed in a future version, please use 'BME' instead.
  pd.date_range("2000-01-01", "2000-12-01", freq="BM")


DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BME')

In [56]:
pd.date_range("2000-01-01", "2000-12-01", freq="BME")

DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31', '2000-04-28',
               '2000-05-31', '2000-06-30', '2000-07-31', '2000-08-31',
               '2000-09-29', '2000-10-31', '2000-11-30'],
              dtype='datetime64[ns]', freq='BME')

**Alias** | `Tipo de desfase` | Descripción

**D** `Day`  Cada día natural

**B** `BusinessDay` Cada día laborable

**H** `Hour` Cada hora

**T o min** `Minute` Una vez por minuto

**S** `Second` Una vez por segundo

**L o ms** `Milli` Milisegundo (1/1.000 de segundo)

**U** `Micro` Microsegundo (1/1.000.000 de segundo)

**M** `MonthEnd` Último día natural del mes

**BM** `BusinessMonthEnd` Último día laborable del mes

**MS** `MonthBegin` Primer día natural del mes

**BMS** `BusinessMonthBegin` Primer día laborable del mes

**W-MON, W-TUE...** `Week` Semanalmente en un determinado día de la semana (MON para lunes, TUE para martes, WED para miércoles, THU para jueves, FRI para viernes, SAT para sábado o SUN para domingo).

**WOM-1MOM, WOM-2MON...** `WeekOfMonth` Genera fechas semanalmente en la primera, segunda, tercera, o cuarta semana del mes (por ejemplo, WOM-3FRI para el tercer viernes de cada mes).

**Q-JAN, Q-FEB...** `QuarterEnd` Fechas trimestrales ancladas en el último día natural de cada mes, por
año que termina en el mes indicado (JAN para enero, FEB para febrero,
MAR para marzo, APR para abril, MAY para mayo, JUN para julio,
JUL para julio, AUG para agosto, SEP para septiembre, OCT para
octubre, NOV para noviembre o DEC para diciembre).

**QS-JAN, QS-FEB** `QuarterBegin` Fechas trimestrales ancladas en el primer día laborable de cada mes,
por año que termina en el mes indicado.

**BQS-JAN, BQS-FEB,...** `BusinessQuarterBegin` Fechas trimestrales ancladas en el primer día laborable de cada mes,
por año que termina en el mes indicado.

**A-JAN, A-FEB,...** `YearEnd` Fechas anuales ancladas en el último día natural del mes dado (JAN
para enero, FEB para febrero, MAR para marzo, APR para abril, MAY
para mayo, JUN para julio, JUL para julio, AUG para agosto, SEP para
septiembre, OCT para octubre, NOV para noviembre o DEC para
diciembre).

**BA-JAN,BA-FEB,...** `BusinessYearEnd` Fechas anuales ancladas en el último día laborable del mes dado.

**AS-JAN, AS-FEB,...** `YearBegin` Fechas anuales ancladas en el primer día del mes dado.

**BAS-JAN, BAS-FEB,...** `BusinessYearBegin` Fechas anuales ancladas en el primer día laborable del mes dado.

`pandas.date_range` conserva por defecto la hora (si la hay) del inicio o fin de la marca temporal:

In [57]:
pd.date_range("2012-05-02 12:56:31", periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

En ocasiones tendremos fechas de inicio o fin con información de hora, pero
nos interesará generar un conjunto de marcas temporales normalizadas a
medianoche como convenio. Para ello existe una opción `normalize`:

In [58]:
pd.date_range("2012-05-02 12:56:31", periods=5, normalize=True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

**Frecuencias y desfases de fechas**

Las frecuencias en pandas están formadas por una frecuencia base y un
multiplicador. Las frecuencias base se suelen indicar con un alias de cadena de
texto, por ejemplo “M” para una frecuencia mensual o “H” para una frecuencia
horaria.  
Para cada frecuencia base tenemos un objeto al que denominamos
desfase de fecha. Por ejemplo, la frecuencia por horas se puede representar con
la clase `Hour`:

In [59]:
from pandas.tseries.offsets import Hour, Minute
hour = Hour()
hour

<Hour>

Por ejemplo:

In [60]:
ts_1 = pd.Timestamp('2023-07-22 10:25:55')

# Desplazar el Timestamp por una hora
new_ts = ts_1 + Hour()
new_ts

Timestamp('2023-07-22 11:25:55')

In [61]:
print(new_ts)

2023-07-22 11:25:55


Se puede definir un múltiplo de un desfase pasando un entero:

In [62]:
four_hours = Hour(4)
print(four_hours)

<4 * Hours>


In [63]:
ts_2 = pd.Timestamp('2023-07-22 10:25:55')

# Desplazar el Timestamp por una hora
new_ts_1 = ts_2 + Hour(5)
print(new_ts_1)

2023-07-22 15:25:55


En la mayoría de las aplicaciones, nunca tendría que ser necesario crear
explícitamente uno de estos objetos, sino que bastaría con emplear un alias de
cadena de texto como `“H”` o `“4H”`. Poner un entero antes de la frecuencia base
crea un múltiplo:

In [64]:
pd.date_range("2000-01-01", "2000-01-03 23:59", freq="4H")

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\3449897904.py:1: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pd.date_range("2000-01-01", "2000-01-03 23:59", freq="4H")


DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 04:00:00',
               '2000-01-01 08:00:00', '2000-01-01 12:00:00',
               '2000-01-01 16:00:00', '2000-01-01 20:00:00',
               '2000-01-02 00:00:00', '2000-01-02 04:00:00',
               '2000-01-02 08:00:00', '2000-01-02 12:00:00',
               '2000-01-02 16:00:00', '2000-01-02 20:00:00',
               '2000-01-03 00:00:00', '2000-01-03 04:00:00',
               '2000-01-03 08:00:00', '2000-01-03 12:00:00',
               '2000-01-03 16:00:00', '2000-01-03 20:00:00'],
              dtype='datetime64[ns]', freq='4h')

Muchos desfases pueden combinarse sumándolos:

In [65]:
Hour(2) + Minute(30)

<150 * Minutes>

De forma similar, se pueden pasar cadenas de texto de frecuencia, como, por
ejemplo, `“1h30min”`, que se analizarán efectivamente en la misma expresión:

In [66]:
pd.date_range("2000-01-01", periods=10, freq="1h30min")

DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 01:30:00',
               '2000-01-01 03:00:00', '2000-01-01 04:30:00',
               '2000-01-01 06:00:00', '2000-01-01 07:30:00',
               '2000-01-01 09:00:00', '2000-01-01 10:30:00',
               '2000-01-01 12:00:00', '2000-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90min')

Algunas frecuencias describen puntos en el tiempo que no están espaciados
por igual. Por ejemplo, “M” (fin de mes natural) y “BM” (último día laborable del mes) dependen del número de días del mes y, en el último caso, de si el mes termina en fin de semana o no. Estas frecuencias se denominan desfases anclados.

**Fechas de la semana del mes**

Una clase de frecuencia útil es `«semana del mes»`, que empieza por `WOM`
(Week Of Month: semana del mes). Permite obtener fechas como, por ejemplo,
el tercer viernes de cada mes:

In [67]:
monthly_dates = pd.date_range("2012-01-01", "2012-09-01", freq="WOM-3FRI")
list(monthly_dates)

[Timestamp('2012-01-20 00:00:00'),
 Timestamp('2012-02-17 00:00:00'),
 Timestamp('2012-03-16 00:00:00'),
 Timestamp('2012-04-20 00:00:00'),
 Timestamp('2012-05-18 00:00:00'),
 Timestamp('2012-06-15 00:00:00'),
 Timestamp('2012-07-20 00:00:00'),
 Timestamp('2012-08-17 00:00:00')]

**Desplazamiento de los datos (adelantar y retrasar)**

El término «desplazamiento» se refiere a mover datos hacia atrás y hacia
adelante en el tiempo. Tanto los objetos Series como DataFrame tienen un
método `shift` para realizar desplazamientos sencillos hacia delante o hacia
atrás, dejando el índice sin modificar:

In [68]:
ts = pd.Series(np.random.standard_normal(4),
               index=pd.date_range("2000-01-01", periods=4, freq="ME")) # ME: Month End: Fin natural de mes
ts

2000-01-31   -0.066748
2000-02-29    0.838639
2000-03-31   -0.117388
2000-04-30   -0.517795
Freq: ME, dtype: float64

In [69]:
ts.shift(2) 
# Desplaza dos valores hacia adelante

2000-01-31         NaN
2000-02-29         NaN
2000-03-31   -0.066748
2000-04-30    0.838639
Freq: ME, dtype: float64

In [70]:
ts.shift(-2)

2000-01-31   -0.117388
2000-02-29   -0.517795
2000-03-31         NaN
2000-04-30         NaN
Freq: ME, dtype: float64

Como los desplazamientos no conscientes de la zona horaria dejan el índice
sin modificar, algunos datos se eliminan. Así, si la frecuencia es conocida, se le
puede pasar a shift para adelantar las marcas temporales en lugar de pasar
simplemente los datos:

In [71]:
ts

2000-01-31   -0.066748
2000-02-29    0.838639
2000-03-31   -0.117388
2000-04-30   -0.517795
Freq: ME, dtype: float64

In [72]:
ts.shift(2, freq="ME")

2000-03-31   -0.066748
2000-04-30    0.838639
2000-05-31   -0.117388
2000-06-30   -0.517795
Freq: ME, dtype: float64

También se pueden pasar otras frecuencias, proporcionando así una cierta
flexibilidad en el modo de adelantar y retrasar los datos:

In [73]:
ts

2000-01-31   -0.066748
2000-02-29    0.838639
2000-03-31   -0.117388
2000-04-30   -0.517795
Freq: ME, dtype: float64

In [74]:
ts.shift(3, freq="D")

2000-02-03   -0.066748
2000-03-03    0.838639
2000-04-03   -0.117388
2000-05-03   -0.517795
dtype: float64

In [75]:
# ts.shift(1, freq="90T")
ts.shift(1, freq="90min")

2000-01-31 01:30:00   -0.066748
2000-02-29 01:30:00    0.838639
2000-03-31 01:30:00   -0.117388
2000-04-30 01:30:00   -0.517795
dtype: float64

Aquí la `T` o (`min`) significa minutos. Tengamos en cuenta que el parámetro `freq`
indica en este caso el desfase que se debe aplicar a las marcas temporales, pero no cambia la frecuencia subyacente de los datos, si es que existe.

**Desplazamiento de fechas con desfases**

Los desfases de fecha de pandas se pueden utilizar también con objetos
datetime o Timestamp:

In [76]:
from pandas.tseries.offsets import Day, MonthEnd
now = datetime(2011, 11, 17)
now + 3 * Day()

Timestamp('2011-11-20 00:00:00')

Si se añade un desfase anclado como MonthEnd, el primer incremento
adelantará la fecha a la siguiente según la regla de frecuencia marcada:

In [77]:
now + MonthEnd()

Timestamp('2011-11-30 00:00:00')

In [78]:
now + MonthEnd(2)

Timestamp('2011-12-31 00:00:00')

Los desfases anclados pueden adelantar o atrasar fechas de forma explícita
simplemente empleando sus métodos `rollforward` y `rollback`,
respectivamente:

In [79]:
now

datetime.datetime(2011, 11, 17, 0, 0)

In [80]:
offset = MonthEnd()
offset.rollforward(now) # 

Timestamp('2011-11-30 00:00:00')

`rollforward` ajusta la fecha dada al final del mes actual si la fecha no está ya al final del mes. Si la fecha ya está al final del mes, no realiza ningún ajuste.

In [81]:
now

datetime.datetime(2011, 11, 17, 0, 0)

In [82]:
offset.rollback(now)

Timestamp('2011-10-31 00:00:00')

`rollback` ajusta la fecha dada al final del mes anterior si la fecha no está ya al final del mes actual. Si la fecha ya está al final del mes, no realiza ningún ajuste.

Los defases de fecha se pueden emplear de forma creativa usando estos
métodos con groupby:

In [83]:
ts = pd.Series(np.random.standard_normal(20),
               index=pd.date_range("2000-01-15", periods=20, freq="4D"))
ts

2000-01-15   -0.116696
2000-01-19    2.389645
2000-01-23   -0.932454
2000-01-27   -0.229331
2000-01-31   -1.140330
2000-02-04    0.439920
2000-02-08   -0.823758
2000-02-12   -0.520930
2000-02-16    0.350282
2000-02-20    0.204395
2000-02-24    0.133445
2000-02-28    0.327905
2000-03-03    0.072153
2000-03-07    0.131678
2000-03-11   -1.297459
2000-03-15    0.997747
2000-03-19    0.870955
2000-03-23   -0.991253
2000-03-27    0.151699
2000-03-31    1.266151
Freq: 4D, dtype: float64

In [84]:
ts.groupby(MonthEnd().rollforward).mean()

2000-01-31   -0.005833
2000-02-29    0.015894
2000-03-31    0.150209
dtype: float64

**Imprimir sin agregaciones el agrupamiento anterior** 

#### **Ejercicio 16**

Pregunta 16.1 Crea una serie temporal ts con 20 valores aleatorios y un índice de fechas que comienzan el 1 de enero de 2022, con una frecuencia diaria. Muestra los primeros 5 valores de la serie.

Pregunta 16.2. Desplaza la serie ts 3 días hacia adelante y muestra los primeros 5 valores.

Pregunta 16.3. Desplaza la serie ts 2 días hacia atrás y muestra los primeros 5 valores.

Pregunta 16.4. Crea una nueva serie ts2 que contenga solo los valores de ts correspondientes a los días 10, 15 y 20 de enero de 2022.

Pregunta 16.5. Agrupa la serie ts por el final de cada mes y calcula la media de los valores en cada grupo.

Pregunta 16.6. Crea una serie temporal ts3 con 20 valores aleatorios y un índice de fechas que comienzan el 1 de enero de 2000, con una frecuencia de 4 días. Muestra los primeros 5 valores.

Pregunta 16.7. Agrupa la serie ts3 por el final de cada mes y calcula la suma de los valores en cada grupo.

Pregunta 16.8. Desplaza la serie ts3 5 días hacia adelante y muestra los primeros 5 valores.

Pregunta 16.9. Utiliza el método asfreq para convertir la frecuencia de la serie ts a una frecuencia diaria y rellena los valores faltantes con ceros.

Pregunta 16.10. Crea una serie temporal ts4 con 15 valores aleatorios y un índice de fechas que comienzan el 15 de enero de 2020, con una frecuencia semanal. Muestra los primeros 5 valores.

Pregunta 16.11. Agrupa la serie ts4 por el final de cada mes y calcula la media de los valores en cada grupo.

Pregunta 16.12. Crea una serie temporal ts5 con 10 valores aleatorios y un índice de fechas que comienzan el 1 de junio de 2021, con una frecuencia de 2 días. Muestra los primeros 5 valores.

Pregunta 16.13.Desplaza la serie ts5 1 semana hacia atrás y muestra los primeros 5 valores.

Pregunta 16.14. Utiliza el método resample para cambiar la frecuencia de la serie ts5 a una frecuencia semanal y muestra los valores resultantes.

Pregunta 16.15.Crea una serie temporal ts6 con 12 valores aleatorios y un índice de fechas que comienzan el 1 de enero de 2019, con una frecuencia mensual. Muestra los primeros 5 valores.

Pregunta 16.16. Desplaza la serie ts6 2 meses hacia adelante y muestra los primeros 5 valores.

Pregunta 16.17. Agrupa la serie ts6 por el final de cada trimestre y calcula la media de los valores en cada grupo.

Pregunta 16.18. Utiliza el método truncate para recortar la serie ts6 y quedarte solo con los valores hasta el final de junio de 2019.

Pregunta 16.19. Crea una serie temporal ts7 con 10 valores aleatorios y un índice de fechas que comienzan el 1 de enero de 2018, con una frecuencia de 3 días. Muestra los primeros 5 valores.

Pregunta 16.20. Agrupa la serie ts7 por el final de cada mes y calcula la suma de los valores en cada grupo.

## 5.4 Manipulación de zonas horarias

El trabajo con zonas horarias puede ser una de las partes menos agradables de
la manipulación de series temporales. Por esta razón muchos usuarios de series
temporales eligen trabajar con ellas en tiempo UTC, o tiempo coordinado
universal (Coordinated Universal Time), el estándar internacional
geográficamente independiente. Las zonas horarias se expresan como desfases
partiendo del estándar UTC; por ejemplo, Nueva York va cuatro horas atrasada
con respecto al tiempo UTC durante el horario de verano y cinco horas el resto
del año.

En Python, la información sobre zonas horarias procede de la librería externa
`pytz`, que expone la base de datos Olson, una compilación de información sobre las zonas horarias mundiales. Esto es de especial importancia en caso de datos
históricos, dado que las fechas de transición de verano e invierno (e inclusos los
desfases UTC) han sido modificados varias veces dependiendo de las leyes
regionales. En los Estados Unidos, los horarios de verano e invierno han
cambiado muchas veces desde 1900.

In [85]:
import pytz

In [86]:
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

Para obtener un objeto de zona horaria de pytz usamos `pytz.timezone`:

In [87]:
tz = pytz.timezone('America/New_York')
tz

<DstTzInfo 'America/New_York' LMT-1 day, 19:04:00 STD>

In [88]:
tiz= pytz.timezone('Europe/Madrid')
tiz

<DstTzInfo 'Europe/Madrid' LMT-1 day, 23:45:00 STD>

Los métodos de pandas aceptarán los nombres de las zonas horarias o estos
objetos.

**Localización y conversión de zonas horarias**

De forma predeterminada, las series temporales de pandas no son conscientes
de las zonas horarias. Por ejemplo, veamos la siguiente serie temporal:

In [89]:
dates = pd.date_range("2012-03-09 09:30", periods=6)
ts = pd.Series(np.random.standard_normal(len(dates)), index=dates)
ts

2012-03-09 09:30:00   -0.202469
2012-03-10 09:30:00    0.050718
2012-03-11 09:30:00    0.639869
2012-03-12 09:30:00    0.597594
2012-03-13 09:30:00   -0.797246
2012-03-14 09:30:00    0.472879
Freq: D, dtype: float64

El campo `tz` del índice es None:

In [90]:
print(ts.index.tz)

None


Se pueden generar rangos de fechas con una zona horaria fijada:

In [91]:
pd.date_range("2012-03-09 09:30", periods=10, tz="UTC")

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

La conversión de no consciente a localizado (reinterpretada como que se ha
observado en una determinada zona horaria) se gestiona con el método
`tz_localize`:

In [92]:
ts

2012-03-09 09:30:00   -0.202469
2012-03-10 09:30:00    0.050718
2012-03-11 09:30:00    0.639869
2012-03-12 09:30:00    0.597594
2012-03-13 09:30:00   -0.797246
2012-03-14 09:30:00    0.472879
Freq: D, dtype: float64

In [93]:
ts_utc = ts.tz_localize("UTC")
ts_utc

2012-03-09 09:30:00+00:00   -0.202469
2012-03-10 09:30:00+00:00    0.050718
2012-03-11 09:30:00+00:00    0.639869
2012-03-12 09:30:00+00:00    0.597594
2012-03-13 09:30:00+00:00   -0.797246
2012-03-14 09:30:00+00:00    0.472879
Freq: D, dtype: float64

In [94]:
ts_utc.index

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

Una vez que una serie se ha localizado en una determinada zona horaria, se
puede convertir a otra con `tz_convert`:

In [95]:
ts_utc.tz_convert("America/New_York")

2012-03-09 04:30:00-05:00   -0.202469
2012-03-10 04:30:00-05:00    0.050718
2012-03-11 05:30:00-04:00    0.639869
2012-03-12 05:30:00-04:00    0.597594
2012-03-13 05:30:00-04:00   -0.797246
2012-03-14 05:30:00-04:00    0.472879
Freq: D, dtype: float64

En el caso de la serie temporal anterior, que está en la transición del horario de verano de la zona horaria `America/New_York`, podríamos localizarla en la
hora del este de EE. UU y convertirla, por ejemplo, a tiempo UTC o a la hora de
Berlín:

In [96]:
ts_eastern = ts.tz_localize("America/New_York")
ts_eastern.tz_convert("UTC")

2012-03-09 14:30:00+00:00   -0.202469
2012-03-10 14:30:00+00:00    0.050718
2012-03-11 13:30:00+00:00    0.639869
2012-03-12 13:30:00+00:00    0.597594
2012-03-13 13:30:00+00:00   -0.797246
2012-03-14 13:30:00+00:00    0.472879
dtype: float64

In [97]:
ts_eastern.tz_convert("Europe/Berlin")

2012-03-09 15:30:00+01:00   -0.202469
2012-03-10 15:30:00+01:00    0.050718
2012-03-11 14:30:00+01:00    0.639869
2012-03-12 14:30:00+01:00    0.597594
2012-03-13 14:30:00+01:00   -0.797246
2012-03-14 14:30:00+01:00    0.472879
dtype: float64

`tz_localize` y `tz_convert` son también métodos de instancia en
`DatetimeIndex`:

In [98]:
ts.index.tz_localize("Asia/Shanghai")

DatetimeIndex(['2012-03-09 09:30:00+08:00', '2012-03-10 09:30:00+08:00',
               '2012-03-11 09:30:00+08:00', '2012-03-12 09:30:00+08:00',
               '2012-03-13 09:30:00+08:00', '2012-03-14 09:30:00+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', freq=None)

**Operaciones con objetos de marca temporal conscientes de la zona horaria**

Más o menos como con las series temporales y los rangos de fechas, los
objetos Timestamp individuales pueden transformarse de no conscientes a
conscientes de la zona horaria y ser convertidos de una zona horaria a otra:

In [99]:
stamp = pd.Timestamp("2011-03-12 04:00")
stamp_utc = stamp.tz_localize("utc")
stamp_utc.tz_convert("America/New_York")

Timestamp('2011-03-11 23:00:00-0500', tz='America/New_York')

También se puede pasar una zona horaria al crear el objeto Timestamp:

In [100]:
stamp_moscow = pd.Timestamp("2011-03-12 04:00", tz="Europe/Moscow")
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

Los objetos Timestamp conscientes de la zona horaria almacenan
internamente un valor de marca temporal UTC como nanosegundos desde el
epoch de Unix (1 de enero de 1970), de modo que cambiar la zona horaria no
altera el valor UTC interno:

In [101]:
stamp_utc.value

1299902400000000000

In [102]:
stamp_utc.tz_convert("America/New_York").value

1299902400000000000

Cuando se realizan operaciones aritméticas temporales utilizando los objetos
DateOffset de pandas, pandas respeta las transiciones de horario de
verano/invierno todo lo posible. En este caso vamos a construir marcas
temporales que ocurren justo antes que las transiciones (hacia delante y hacia
atrás). Primero, 30 minutos antes de cambiar a la hora de verano:

In [103]:
stamp = pd.Timestamp("2012-03-11 01:30", tz="US/Eastern")
stamp

Timestamp('2012-03-11 01:30:00-0500', tz='US/Eastern')

In [104]:
stamp + Hour()

Timestamp('2012-03-11 03:30:00-0400', tz='US/Eastern')

Después, 90 minutos antes de cambiar al horario de invierno:

In [105]:
stamp = pd.Timestamp("2012-11-04 00:30", tz="US/Eastern")
stamp

Timestamp('2012-11-04 00:30:00-0400', tz='US/Eastern')

In [106]:
stamp + 2 * Hour()

Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

**Operaciones entre distintas zonas horarias**

Si dos series temporales con distintas zonas horarias se combinan, el
resultado será UTC. Como las marcas temporales se almacenan internamente en
UTC, es una operación directa y no requiere conversión:

In [107]:
dates = pd.date_range("2012-03-07 09:30", periods=10, freq="B")
ts = pd.Series(np.random.standard_normal(len(dates)), index=dates)
ts

2012-03-07 09:30:00    0.522356
2012-03-08 09:30:00   -0.546348
2012-03-09 09:30:00   -0.733537
2012-03-12 09:30:00    1.302736
2012-03-13 09:30:00    0.022199
2012-03-14 09:30:00    0.364287
2012-03-15 09:30:00   -0.922839
2012-03-16 09:30:00    0.312656
2012-03-19 09:30:00   -1.128497
2012-03-20 09:30:00   -0.333488
Freq: B, dtype: float64

In [108]:
ts1 = ts[:7].tz_localize("Europe/London")
ts1

2012-03-07 09:30:00+00:00    0.522356
2012-03-08 09:30:00+00:00   -0.546348
2012-03-09 09:30:00+00:00   -0.733537
2012-03-12 09:30:00+00:00    1.302736
2012-03-13 09:30:00+00:00    0.022199
2012-03-14 09:30:00+00:00    0.364287
2012-03-15 09:30:00+00:00   -0.922839
dtype: float64

In [109]:
ts2 = ts1[2:].tz_convert("Europe/Moscow")
ts2

2012-03-09 13:30:00+04:00   -0.733537
2012-03-12 13:30:00+04:00    1.302736
2012-03-13 13:30:00+04:00    0.022199
2012-03-14 13:30:00+04:00    0.364287
2012-03-15 13:30:00+04:00   -0.922839
dtype: float64

In [110]:
result = ts1 + ts2
result

2012-03-07 09:30:00+00:00         NaN
2012-03-08 09:30:00+00:00         NaN
2012-03-09 09:30:00+00:00   -1.467074
2012-03-12 09:30:00+00:00    2.605472
2012-03-13 09:30:00+00:00    0.044397
2012-03-14 09:30:00+00:00    0.728575
2012-03-15 09:30:00+00:00   -1.845677
dtype: float64

In [111]:
result.index

DatetimeIndex(['2012-03-07 09:30:00+00:00', '2012-03-08 09:30:00+00:00',
               '2012-03-09 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq=None)

## 5.5 Periodos y aritmética de periodos

Los periodos representan lapsos de tiempo, como días, meses, trimestres o
años. La clase `pandas.Period` representa este tipo de datos, y requiere una cadena de texto o un entero y una frecuencia soportada por pandas.

In [112]:
p = pd.Period("2011", freq="A-DEC")
p

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\520127536.py:1: FutureWarning: 'A-DEC' is deprecated and will be removed in a future version, please use 'Y-DEC' instead.
  p = pd.Period("2011", freq="A-DEC")


Period('2011', 'Y-DEC')

En este caso, el objeto Period representa el lapso de tiempo completo desde
el 1 de enero de 2011 hasta el 31 de diciembre de 2011, inclusive. Resulta
cómodo que sumar y restar enteros a los periodos tenga como efecto desplazar
su frecuencia:

In [113]:
p + 5

Period('2016', 'Y-DEC')

In [114]:
p - 2

Period('2009', 'Y-DEC')

Si dos periodos tienen la misma frecuencia, su diferencia es el número de
unidades entre ellos como desfase de fecha:

In [115]:
pd.Period("2014", freq="A-DEC") - p

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\3816979827.py:1: FutureWarning: 'A-DEC' is deprecated and will be removed in a future version, please use 'Y-DEC' instead.
  pd.Period("2014", freq="A-DEC") - p


<3 * YearEnds: month=12>

Se pueden construir rangos regulares de periodos con la función
`period_range`:

In [116]:
periods = pd.period_range("2000-01-01", "2000-06-30", freq="M")
periods

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]')

La clase PeriodIndex almacena una secuencia de periodos y puede servir
como índice de eje en cualquier estructura de datos de pandas:

In [117]:
pd.Series(np.random.standard_normal(6), index=periods)

2000-01   -0.514551
2000-02   -0.559782
2000-03   -0.783408
2000-04   -1.797685
2000-05   -0.172670
2000-06    0.680215
Freq: M, dtype: float64

Si tenemos un array de cadenas de texto, también podemos usar la clase
PeriodIndex, cuyos valores son todos periodos:

In [118]:
values = ["2001Q3", "2002Q2", "2003Q1"]
index = pd.PeriodIndex(values, freq="Q-DEC")
index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]')

**Conversión de frecuencias de periodos**

Los periodos y los objetos PeriodIndex se pueden convertir a otra frecuencia
con su método `asfreq`. Como ejemplo, supongamos que tenemos un periodo
anual y queremos convertirlo a mensual al principio o final del año. Esto puede
hacerse así:

In [119]:
p = pd.Period("2011", freq="A-DEC")
p

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\520127536.py:1: FutureWarning: 'A-DEC' is deprecated and will be removed in a future version, please use 'Y-DEC' instead.
  p = pd.Period("2011", freq="A-DEC")


Period('2011', 'Y-DEC')

In [120]:
p.asfreq("M", how="start")

Period('2011-01', 'M')

In [121]:
p.asfreq("M", how="end")

Period('2011-12', 'M')

In [122]:
p.asfreq("M")

Period('2011-12', 'M')

Se puede pensar en `Period(“2011”, “A-DEC”)` como si fuera una especie de
cursor apuntando a un lapso de tiempo, subdividido en periodos mensuales. La figura 5.1 muestra una ilustración de ello. En el caso de un año fiscal que
termine en un mes distinto a diciembre, los subperiodos mensuales
correspondientes son distintos:

In [123]:
p = pd.Period("2011", freq="A-JUN")
p

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\3767667261.py:1: FutureWarning: 'A-JUN' is deprecated and will be removed in a future version, please use 'Y-JUN' instead.
  p = pd.Period("2011", freq="A-JUN")


Period('2011', 'Y-JUN')

In [124]:
p.asfreq("M", how="start")

Period('2010-07', 'M')

In [125]:
p.asfreq("M", how="end")

Period('2011-06', 'M')

<img src="5.1.png">

Cuando estamos convirtiendo de frecuencia alta a baja, pandas determina el
subperiodo, dependiendo de donde «pertenezca» el superperiodo. Por ejemplo,
en una frecuencia A-JUN, el mes Aug-2011 es en realidad parte del periodo 2012:

In [126]:
p = pd.Period("Aug-2011", "M")
p.asfreq("A-JUN")

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\2271097004.py:2: FutureWarning: 'A-JUN' is deprecated and will be removed in a future version, please use 'Y-JUN' instead.
  p.asfreq("A-JUN")


Period('2012', 'Y-JUN')

Se pueden convertir objetos o series temporales PeriodIndex enteros de una
forma parecida con la misma semántica:

In [127]:
periods = pd.period_range("2006", "2009", freq="A-DEC")
ts = pd.Series(np.random.standard_normal(len(periods)), index=periods)
ts

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\2389045503.py:1: FutureWarning: 'A-DEC' is deprecated and will be removed in a future version, please use 'Y-DEC' instead.
  periods = pd.period_range("2006", "2009", freq="A-DEC")


2006    1.607578
2007    0.200381
2008   -0.834068
2009   -0.302988
Freq: Y-DEC, dtype: float64

In [128]:
ts.asfreq("M", how="start")

2006-01    1.607578
2007-01    0.200381
2008-01   -0.834068
2009-01   -0.302988
Freq: M, dtype: float64

Aquí, los periodos anuales son reemplazados por periodos mensuales que
corresponden al primer mes que caiga dentro de cada periodo anual. Pero si
queremos el último día laborable de cada año, podemos usar la frecuencia “B” e
indicar que queremos el final del periodo:

In [129]:
ts.asfreq("B", how="end")

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\2984082239.py:1: FutureWarning: PeriodDtype[B] is deprecated and will be removed in a future version. Use a DatetimeIndex with freq='B' instead
  ts.asfreq("B", how="end")


2006-12-29    1.607578
2007-12-31    0.200381
2008-12-31   -0.834068
2009-12-31   -0.302988
Freq: B, dtype: float64

**Frecuencias de periodos trimestrales**

Los datos trimestrales son estándares en contabilidad, finanzas y otros
campos. Gran parte de estos datos se comunican en relación a un cierre de año
fiscal, normalmente el último día natural o laborable de uno de los 12 meses del
año. Es decir, el periodo `2012Q4` tiene un significado distinto dependiendo del
final del año fiscal. En pandas se soportan las 12 frecuencias trimestrales
posibles desde `Q-JAN` hasta `Q-DEC`:

In [130]:
p = pd.Period("2012Q4", freq="Q-JAN")
p

Period('2012Q4', 'Q-JAN')

En el caso de un año fiscal que termine en enero, 2012Q4 va desde noviembre
de 2011 hasta enero de 2012, lo que puede verificarse convirtiendo a frecuencia
diaria

In [131]:
p.asfreq("D", how="start")

Period('2011-11-01', 'D')

In [132]:
p.asfreq("D", how="end")

Period('2012-01-31', 'D')

Se pueden generar rangos trimestrales empleando `pandas.period_range`.
Los cálculos son también idénticos:

In [133]:
periods = pd.period_range("2011Q3", "2012Q4", freq="Q-JAN")
ts = pd.Series(np.arange(len(periods)), index=periods)
ts

2011Q3    0
2011Q4    1
2012Q1    2
2012Q2    3
2012Q3    4
2012Q4    5
Freq: Q-JAN, dtype: int64

**Conversión de marcas temporales a periodos (y viceversa)**

Los objetos Series y DataFrame indexados por series temporales pueden
convertirse a periodos con el método `to_period`:

In [134]:
dates = pd.date_range("2000-01-01", periods=3, freq="M")
dates

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\1325953892.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range("2000-01-01", periods=3, freq="M")


DatetimeIndex(['2000-01-31', '2000-02-29', '2000-03-31'], dtype='datetime64[ns]', freq='ME')

In [135]:
ts = pd.Series(np.random.standard_normal(3), index=dates)
ts

2000-01-31    1.663261
2000-02-29   -0.996206
2000-03-31    1.521760
Freq: ME, dtype: float64

In [136]:
pts = ts.to_period()
pts

2000-01    1.663261
2000-02   -0.996206
2000-03    1.521760
Freq: M, dtype: float64

Como los periodos hacen referencia a lapsos de tiempo que no se superponen,
una marca temporal solo puede pertenecer a un único periodo para una
determinada frecuencia. Aunque la frecuencia del nuevo PeriodIndex se puede
deducir de las marcas temporales de forma predeterminada, es posible
especificar cualquier frecuencia soportada (la mayoría de las listadas en la tabla
5.4 se soportan). Tampoco es un problema que aparezcan periodos duplicados
en el resultado:

In [137]:
dates = pd.date_range("2000-01-29", periods=6)
dates

DatetimeIndex(['2000-01-29', '2000-01-30', '2000-01-31', '2000-02-01',
               '2000-02-02', '2000-02-03'],
              dtype='datetime64[ns]', freq='D')

In [138]:
ts2 = pd.Series(np.random.standard_normal(6), index=dates)
ts2

2000-01-29    0.244175
2000-01-30    0.423331
2000-01-31   -0.654040
2000-02-01    2.089154
2000-02-02   -0.060220
2000-02-03   -0.167933
Freq: D, dtype: float64

In [139]:
ts2.to_period("M")

2000-01    0.244175
2000-01    0.423331
2000-01   -0.654040
2000-02    2.089154
2000-02   -0.060220
2000-02   -0.167933
Freq: M, dtype: float64

Para volver a convertir a marcas temporales, usamos el método
`to_timestamp`, que devuelve un objeto DatetimeIndex:

In [140]:
pts = ts2.to_period()
pts

2000-01-29    0.244175
2000-01-30    0.423331
2000-01-31   -0.654040
2000-02-01    2.089154
2000-02-02   -0.060220
2000-02-03   -0.167933
Freq: D, dtype: float64

In [141]:
pts.to_timestamp(how="end")

2000-01-29 23:59:59.999999999    0.244175
2000-01-30 23:59:59.999999999    0.423331
2000-01-31 23:59:59.999999999   -0.654040
2000-02-01 23:59:59.999999999    2.089154
2000-02-02 23:59:59.999999999   -0.060220
2000-02-03 23:59:59.999999999   -0.167933
Freq: D, dtype: float64

**Creación de un objeto PeriodIndex a partir de arrays**

Los conjuntos de datos de frecuencia fija se almacenan en ocasiones con
información de lapso temporal distribuida a lo largo de varias columnas. Por
ejemplo, en este conjunto de datos macroeconómico, el año y el trimestre están
en columnas distintas:

In [142]:
data = pd.read_csv("macrodata.xls")
data.head(5)

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [143]:
data["year"]

0      1959
1      1959
2      1959
3      1959
4      1960
       ... 
198    2008
199    2008
200    2009
201    2009
202    2009
Name: year, Length: 203, dtype: int64

In [144]:
data["quarter"]

0      1
1      2
2      3
3      4
4      1
      ..
198    3
199    4
200    1
201    2
202    3
Name: quarter, Length: 203, dtype: int64

Pasando estos arrays a PeriodIndex con una frecuencia, se pueden combinar
para formar un índice para el dataframe:

In [145]:
index = pd.PeriodIndex(year=data["year"], quarter=data["quarter"],
                       freq="Q-DEC")
index

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\936327627.py:1: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  index = pd.PeriodIndex(year=data["year"], quarter=data["quarter"],


PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', length=203)

In [146]:
data.index = index

In [147]:
data["infl"]

1959Q1    0.00
1959Q2    2.34
1959Q3    2.74
1959Q4    0.27
1960Q1    2.31
          ... 
2008Q3   -3.16
2008Q4   -8.79
2009Q1    0.94
2009Q2    3.37
2009Q3    3.56
Freq: Q-DEC, Name: infl, Length: 203, dtype: float64

**Remuestreo y conversión de frecuencias**

El término remuestreo se refiere al proceso de convertir una serie temporal de
una frecuencia a otra. Al proceso de agregar datos de alta frecuencia a baja
frecuencia se le denomina submuestreo, mientras que al proceso contrario se le
llama sobremuestreo.

No todos los remuestreos entran en estas dos
categorías; por ejemplo, convertir `W-WED` (semanalmente los miércoles) en `W-FRI`
no es ni sobremuestreo ni submuestreo.

Los objetos pandas van equipados con un método `resample`, la función
principal para todos los tipos de conversión de frecuencias. Este método tiene
una API similar a `groupby`; primero llamamos a `resample` para agrupar los
datos, y después llamamos a una función de agregación:

In [148]:
dates = pd.date_range("2000-01-01", periods=100)
ts = pd.Series(np.random.standard_normal(len(dates)), index=dates)
ts

2000-01-01    0.631634
2000-01-02   -1.594313
2000-01-03   -1.519937
2000-01-04    1.108752
2000-01-05    1.255853
                ...   
2000-04-05   -0.423776
2000-04-06    0.789740
2000-04-07    0.937568
2000-04-08   -2.253294
2000-04-09   -1.772919
Freq: D, Length: 100, dtype: float64

In [149]:
ts.resample("M").mean()

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\3978117481.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ts.resample("M").mean()


2000-01-31   -0.165893
2000-02-29    0.078606
2000-03-31    0.223811
2000-04-30   -0.063643
Freq: ME, dtype: float64

In [150]:
ts.resample("M", kind="period").mean()
# El tipo de período kind="period"
# indica a Pandas que debe resamplear 
# la serie de datos en función de períodos calendario,
# en lugar de frecuencias basadas 
# en el tiempo (como segundos, minutos u horas). 

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\1516376471.py:1: FutureWarning: The 'kind' keyword in Series.resample is deprecated and will be removed in a future version. Explicitly cast the index to the desired type instead
  ts.resample("M", kind="period").mean()
C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\1516376471.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  ts.resample("M", kind="period").mean()


2000-01   -0.165893
2000-02    0.078606
2000-03    0.223811
2000-04   -0.063643
Freq: M, dtype: float64

La siguiente tabla 5.5 resume algunas opciones del métod resample

**Argumento** | **Descripción**

`rule`: Cadena de texto, DateOffset o timedelta que indica la frecuencia de remuestreo deseada.(por ejemplo, "M", "5min" o Second(15)).

`axis`: Eje según el cual remuestrear; por defecto es axis=0.

`closed`: En submuestreos, qué extremo de cada intervalo está cerrado (inclusive), "right" o "left".

`label` : En submuestreos, cómo etiquetar el resultado agregado, con el borde de contenedor
"right" o "left" (por ejemplo, el intervalo de cinco minutos de 9:30 a 9:35 se puede
etiquetar como 9:30 o 9:35).

`limit` : Cuando se rellena hacia delante o hacia atrás, el número máximo de periodos que se van a rellenar.

`kind` : Agrega a periodos ("period") o marcas temporales ("timestamp"); el valor predeterminado es el tipo de índice que tiene la serie temporal.

`convention`: Cuando se remuestrean periodos, el convenio ("start" o "end") para convertir el periodo de baja frecuencia en alta frecuencia; por defecto es "start". 
 `origin`: La marca temporal «base» a partir de la cual se determinan los bordes de contenedor de
remuestreo; también puede ser "epoch", "start", "start_day", "end" o "end_day". En el docstring de resample se dispone de más información.

`offset` : Un timedelta de desfase añadido al origen; su valor predeterminado es None.





**Submuestreo**

Submuestrear es agregar datos a una frecuencia baja regular. Hay un par de cosas en las que conviene pensar cuando se
emplea resample para submuestrear datos:

- Qué lado de cada intervalo está cerrado.

- Cómo etiquetar cada contenedor agregado, ya sea con el inicio del intervalo o con el final.

A modo de ilustración, veamos unos datos de frecuencia de un minuto:

In [151]:
dates = pd.date_range("2000-01-01", periods=12, freq="T")
dates

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\3047982846.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  dates = pd.date_range("2000-01-01", periods=12, freq="T")


DatetimeIndex(['2000-01-01 00:00:00', '2000-01-01 00:01:00',
               '2000-01-01 00:02:00', '2000-01-01 00:03:00',
               '2000-01-01 00:04:00', '2000-01-01 00:05:00',
               '2000-01-01 00:06:00', '2000-01-01 00:07:00',
               '2000-01-01 00:08:00', '2000-01-01 00:09:00',
               '2000-01-01 00:10:00', '2000-01-01 00:11:00'],
              dtype='datetime64[ns]', freq='min')

In [152]:
ts = pd.Series(np.arange(len(dates)), index=dates)
ts

2000-01-01 00:00:00     0
2000-01-01 00:01:00     1
2000-01-01 00:02:00     2
2000-01-01 00:03:00     3
2000-01-01 00:04:00     4
2000-01-01 00:05:00     5
2000-01-01 00:06:00     6
2000-01-01 00:07:00     7
2000-01-01 00:08:00     8
2000-01-01 00:09:00     9
2000-01-01 00:10:00    10
2000-01-01 00:11:00    11
Freq: min, dtype: int64

Supongamos que queremos agregar estos datos en fragmentos o barras de
cinco minutos tomando la suma de cada grupo:

In [153]:
ts.resample("5min").sum()

2000-01-01 00:00:00    10
2000-01-01 00:05:00    35
2000-01-01 00:10:00    21
Freq: 5min, dtype: int64

La frecuencia que se pasa define los bordes del contenedor en incrementos de
cinco minutos. Para esta frecuencia, el borde izquierdo del contenedor es
inclusivo por defecto, de modo que el valor 00:00 está incluido en el intervalo
00:00 a 00:05, y el valor 00:05 está excluido de dicho intervalo

In [154]:
ts.resample("5min", closed="right").sum()

1999-12-31 23:55:00     0
2000-01-01 00:00:00    15
2000-01-01 00:05:00    40
2000-01-01 00:10:00    11
Freq: 5min, dtype: int64

La serie resultante es etiquetada por las marcas temporales desde el lado
izquierdo de cada contenedor. Pasando `label=”right”` se pueden etiquetar con
el borde derecho del contenedor:

In [155]:
ts.resample("5min", closed="right", label="right").sum()

2000-01-01 00:00:00     0
2000-01-01 00:05:00    15
2000-01-01 00:10:00    40
2000-01-01 00:15:00    11
Freq: 5min, dtype: int64

Por último, puede que queramos desplazar el índice del resultado en una
cierta cantidad, digamos restando un segundo desde el contenedor derecho para
dejar más claro a qué intervalo se refiere la marca temporal. Para hacer esto,
añadimos un desfase al índice resultante:

In [156]:
from pandas.tseries.frequencies import to_offset
result = ts.resample("5min", closed="right", label="right").sum()
result

2000-01-01 00:00:00     0
2000-01-01 00:05:00    15
2000-01-01 00:10:00    40
2000-01-01 00:15:00    11
Freq: 5min, dtype: int64

In [157]:
result.index = result.index + to_offset("-1s")
result

1999-12-31 23:59:59     0
2000-01-01 00:04:59    15
2000-01-01 00:09:59    40
2000-01-01 00:14:59    11
Freq: 5min, dtype: int64

**Remuestreo OHLC (Open-high-low-close)**

En finanzas, una forma habitual de agregar una serie temporal es calcular
cuatro valores para cada contenedor: el primero (open: abrir), el último (close:
cerrar), el máximo (high: alto) y el mínimo (low: bajo). Utilizando la función de
agregación `ohlc` obtendremos un dataframe cuyas columnas contienen estos
cuatro agregados, que se calculan de forma eficaz en una única llamada a
función:

In [158]:
ts = pd.Series(np.random.permutation(np.arange(len(dates))), index=dates)
ts

2000-01-01 00:00:00     8
2000-01-01 00:01:00     3
2000-01-01 00:02:00     1
2000-01-01 00:03:00     4
2000-01-01 00:04:00     5
2000-01-01 00:05:00     6
2000-01-01 00:06:00    10
2000-01-01 00:07:00    11
2000-01-01 00:08:00     9
2000-01-01 00:09:00     2
2000-01-01 00:10:00     0
2000-01-01 00:11:00     7
Freq: min, dtype: int64

In [159]:
ts.resample("5min").ohlc()

,open,high,low,close
2000-01-01 00:00:00,8,8,1,5
2000-01-01 00:05:00,6,11,2,2
2000-01-01 00:10:00,0,7,0,7


**Sobremuestreo e interpolación**

Sobremuestrear es convertir de una frecuencia baja a otra alta, donde no se
necesita agregación. Veamos un dataframe con algunos datos semanales:

In [160]:
frame = pd.DataFrame(np.random.standard_normal((2, 4)),
                     index=pd.date_range("2000-01-01", periods=2,
                                         freq="W-WED"),
                     columns=["Colorado", "Texas", "New York", "Ohio"])
frame

,Colorado,Texas,New York,Ohio
2000-01-05,-0.896431,0.927238,0.482284,-0.867130
2000-01-12,0.493841,-0.155434,1.397286,1.507055


 Empleamos el método `asfreq` para convertir a la frecuencia más alta sin agregación alguna:

In [161]:
df_daily = frame.resample("D").asfreq()
df_daily

,Colorado,Texas,New York,Ohio
2000-01-05,-0.896431,0.927238,0.482284,-0.867130
2000-01-06,NaN,NaN,NaN,NaN
2000-01-07,NaN,NaN,NaN,NaN
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,0.493841,-0.155434,1.397286,1.507055


Supongamos que queremos rellenar hacia adelante todos los valores
semanales que no sean miércoles. Los mismos métodos de rellenado o
interpolación que están disponibles en los métodos `fillna` y `reindex` lo están disponibles
para remuestrear:

In [162]:
frame.resample("D").ffill()

,Colorado,Texas,New York,Ohio
2000-01-05,-0.896431,0.927238,0.482284,-0.867130
2000-01-06,-0.896431,0.927238,0.482284,-0.867130
2000-01-07,-0.896431,0.927238,0.482284,-0.867130
2000-01-08,-0.896431,0.927238,0.482284,-0.867130
2000-01-09,-0.896431,0.927238,0.482284,-0.867130
2000-01-10,-0.896431,0.927238,0.482284,-0.867130
2000-01-11,-0.896431,0.927238,0.482284,-0.867130
2000-01-12,0.493841,-0.155434,1.397286,1.507055


De forma similar, se puede elegir rellenar solamente un determinado número
de periodos hacia delante para limitar hasta dónde continuar utilizando un valor
observado:

In [163]:
frame.resample("D").ffill(limit=2)

,Colorado,Texas,New York,Ohio
2000-01-05,-0.896431,0.927238,0.482284,-0.867130
2000-01-06,-0.896431,0.927238,0.482284,-0.867130
2000-01-07,-0.896431,0.927238,0.482284,-0.867130
2000-01-08,NaN,NaN,NaN,NaN
2000-01-09,NaN,NaN,NaN,NaN
2000-01-10,NaN,NaN,NaN,NaN
2000-01-11,NaN,NaN,NaN,NaN
2000-01-12,0.493841,-0.155434,1.397286,1.507055


Particularmente se observa que el nuevo índice de fecha no tiene por qué
coincidir con el antiguo:

In [164]:
frame.resample("W-THU").ffill()

,Colorado,Texas,New York,Ohio
2000-01-06,-0.896431,0.927238,0.482284,-0.867130
2000-01-13,0.493841,-0.155434,1.397286,1.507055


**Remuestreo con periodos**

El remuestreo de datos indexados por periodos es similar a las marcas de
tiempo:

In [165]:
frame = pd.DataFrame(np.random.standard_normal((24, 4)),
                     index=pd.period_range("1-2000", "12-2001",
                                           freq="M"),
                     columns=["Colorado", "Texas", "New York", "Ohio"])
frame.head()

,Colorado,Texas,New York,Ohio
2000-01,-1.179442,0.443171,1.395676,-0.529658
2000-02,0.787358,0.248845,0.743239,1.267746
2000-03,1.302395,-0.272154,-0.051532,-0.467740
2000-04,-1.040816,0.426419,0.312945,-1.115689
2000-05,1.234297,-1.893094,-1.661605,-0.005477


In [166]:
annual_frame = frame.resample("A-DEC").mean()
annual_frame

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\3051886458.py:1: FutureWarning: 'A-DEC' is deprecated and will be removed in a future version, please use 'Y-DEC' instead.
  annual_frame = frame.resample("A-DEC").mean()
C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\3051886458.py:1: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  annual_frame = frame.resample("A-DEC").mean()


,Colorado,Texas,New York,Ohio
2000,0.487329,0.104466,0.020495,-0.273945
2001,0.203125,0.162429,0.056146,-0.103794


El sobremuestreo está más matizado, porque antes de muestrear es necesario
tomar una decisión sobre el extremo del lapso de tiempo de la nueva frecuencia
en el que poner los valores. El argumento convention tiene como valor
predeterminado “start” pero también puede ser “end”:

In [167]:
# Q-DEC: Trimestral, el año termina en diciembre
annual_frame.resample("Q-DEC").ffill()

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\206118844.py:2: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  annual_frame.resample("Q-DEC").ffill()


,Colorado,Texas,New York,Ohio
2000Q1,0.487329,0.104466,0.020495,-0.273945
2000Q2,0.487329,0.104466,0.020495,-0.273945
2000Q3,0.487329,0.104466,0.020495,-0.273945
2000Q4,0.487329,0.104466,0.020495,-0.273945
2001Q1,0.203125,0.162429,0.056146,-0.103794
2001Q2,0.203125,0.162429,0.056146,-0.103794
2001Q3,0.203125,0.162429,0.056146,-0.103794
2001Q4,0.203125,0.162429,0.056146,-0.103794


In [168]:
annual_frame.resample("Q-DEC", convention="end").asfreq()

C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\1415943662.py:1: FutureWarning: The 'convention' keyword in DataFrame.resample is deprecated and will be removed in a future version. Explicitly cast PeriodIndex to DatetimeIndex before resampling instead.
  annual_frame.resample("Q-DEC", convention="end").asfreq()
C:\Users\juanj\AppData\Local\Temp\ipykernel_11260\1415943662.py:1: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  annual_frame.resample("Q-DEC", convention="end").asfreq()


,Colorado,Texas,New York,Ohio
2000Q4,0.487329,0.104466,0.020495,-0.273945
2001Q1,NaN,NaN,NaN,NaN
2001Q2,NaN,NaN,NaN,NaN
2001Q3,NaN,NaN,NaN,NaN
2001Q4,0.203125,0.162429,0.056146,-0.103794


**Remuestreo de tiempo agrupado**

Para datos de series temporales, el método resample es semánticamente una
operación de grupo basada en una intervalización de tiempo. Aquí tenemos una
tabla de ejemplo:

In [169]:
N = 15
times = pd.date_range("2017-05-20 00:00", freq="1min", periods=N)
times

DatetimeIndex(['2017-05-20 00:00:00', '2017-05-20 00:01:00',
               '2017-05-20 00:02:00', '2017-05-20 00:03:00',
               '2017-05-20 00:04:00', '2017-05-20 00:05:00',
               '2017-05-20 00:06:00', '2017-05-20 00:07:00',
               '2017-05-20 00:08:00', '2017-05-20 00:09:00',
               '2017-05-20 00:10:00', '2017-05-20 00:11:00',
               '2017-05-20 00:12:00', '2017-05-20 00:13:00',
               '2017-05-20 00:14:00'],
              dtype='datetime64[ns]', freq='min')

In [170]:
df = pd.DataFrame({"time": times,
                   "value": np.arange(N)})
df

,time,value
0,2017-05-20 00:00:00,0
1,2017-05-20 00:01:00,1
2,2017-05-20 00:02:00,2
3,2017-05-20 00:03:00,3
4,2017-05-20 00:04:00,4
5,2017-05-20 00:05:00,5
6,2017-05-20 00:06:00,6
7,2017-05-20 00:07:00,7
8,2017-05-20 00:08:00,8
9,2017-05-20 00:09:00,9


Aquí, podemos indexar por “time” y después remuestrear:

In [171]:
df.set_index("time").resample("5min").count()

,value
time,
2017-05-20 00:00:00,5
2017-05-20 00:05:00,5
2017-05-20 00:10:00,5


Supongamos que un dataframe contiene varias series temporales, marcadas
por una columna de clave de grupo adicional:

In [172]:
df2 = pd.DataFrame({"time": times.repeat(3),
                    "key": np.tile(["a", "b", "c"], N),
                    "value": np.arange(N * 3.)})
df2.head(7)

,time,key,value
0,2017-05-20 00:00:00,a,0.0
1,2017-05-20 00:00:00,b,1.0
2,2017-05-20 00:00:00,c,2.0
3,2017-05-20 00:01:00,a,3.0
4,2017-05-20 00:01:00,b,4.0
5,2017-05-20 00:01:00,c,5.0
6,2017-05-20 00:02:00,a,6.0


In [173]:
df2.tail()

,time,key,value
40,2017-05-20 00:13:00,b,40.0
41,2017-05-20 00:13:00,c,41.0
42,2017-05-20 00:14:00,a,42.0
43,2017-05-20 00:14:00,b,43.0
44,2017-05-20 00:14:00,c,44.0


Para hacer el mismo remuestreo para cada valor de “key”, introducimos el
objeto pandas.Grouper:

In [174]:
time_key = pd.Grouper(freq="5min")

Podemos a continuación establecer el índice de hora, agrupar por “key” y
time_key, y finalmente agregar:

In [175]:
resampled = (df2.set_index("time")
             .groupby(["key", time_key])
             .sum())
resampled

value
key time                      
a   2017-05-20 00:00:00   30.0
    2017-05-20 00:05:00  105.0
    2017-05-20 00:10:00  180.0
b   2017-05-20 00:00:00   35.0
    2017-05-20 00:05:00  110.0
    2017-05-20 00:10:00  185.0
c   2017-05-20 00:00:00   40.0
    2017-05-20 00:05:00  115.0
    2017-05-20 00:10:00  190.0

In [176]:
resampled.reset_index()

,key,time,value
0,a,2017-05-20 00:00:00,30.0
1,a,2017-05-20 00:05:00,105.0
2,a,2017-05-20 00:10:00,180.0
3,b,2017-05-20 00:00:00,35.0
4,b,2017-05-20 00:05:00,110.0
5,b,2017-05-20 00:10:00,185.0
6,c,2017-05-20 00:00:00,40.0
7,c,2017-05-20 00:05:00,115.0
8,c,2017-05-20 00:10:00,190.0
